# NLP Final Project: Authorship Attribution

## Yuval Timen

##### We will be exploring the task of Authorship Attribution - trying to assign to each piece of text the author who wrote it. We will be looking at the works of Aristotle, Friedrich Nietzsche, William Shakespeare, Henry Wadsworth Longfellow, Percy Bysshe Shelley, Winston Churchill (the American author, not the British Prime Minister), and Robert Browning. All data was collected from Project Gutenburg. 

##### We
